In [24]:
import joblib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import os
import re
from unidecode import unidecode

In [25]:
def preprocess(text):
    # Remover acentos
    text = unidecode(text)

    # Substituir caracteres não alfabéticos (incluindo vírgula) por espaços em branco
    text = re.sub(r'[^a-zA-Z]', ' ', text)

    # Converter para minúsculas e dividir em palavras
    tokens = text.lower().split()

    return ' '.join(tokens)

In [26]:
def load_lyrics():
    # Certifique-se de que os dados carregados são textos brutos antes do pré-processamento
    raw_lyrics =  joblib.load('../lyrics_vector.joblib')
    return [preprocess(text) for text in raw_lyrics]



In [27]:
def create_tfidf_matrix(lyrics):
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(lyrics)
    return tfidf_matrix, vectorizer


In [28]:
def perform_query(query, vectorizer, tfidf_matrix, lyrics):
    preprocessed_query = preprocess(query)
    print(f"\nConsulta: {preprocessed_query}")
    query_vector = vectorizer.transform([preprocessed_query])

    # Calcular similaridade
    cosine_similarities = linear_kernel(query_vector, tfidf_matrix).flatten()

    # Adicionar um fator de prioridade para palavras em sequência
    sequence_priority = 100.0  # Ajuste conforme necessário
    for i in range(len(lyrics)):
        document = lyrics[i]
        occurrences = document.lower().count(preprocessed_query)
        cosine_similarities[i] += sequence_priority * occurrences

    document_scores = list(enumerate(cosine_similarities))
    sorted_documents = sorted(document_scores, key=lambda x: x[1], reverse=True)

    # Imprimir resultados
    print("\nResultados:")
    for index, score in sorted_documents[:5]:
        print(
            f"Arquivo: {os.listdir('C:/Users/miguel/Documents/UFV/6 PERIODO/GRADI/musics-txt')[index]}, Pontuação: {score}")


In [29]:

# Carregar vetor de letras
print("Carregando letras...")
lyrics = load_lyrics()

# Criar matriz TF-IDF
print("Criando matriz TF-IDF...")
tfidf_matrix, vectorizer = create_tfidf_matrix(lyrics)
print(f"Dimensões da matriz TF-IDF: {tfidf_matrix.shape}")

Carregando letras...
Criando matriz TF-IDF...
Dimensões da matriz TF-IDF: (379679, 522857)


In [32]:
def main():
    # Menu de inserção de consulta
    while True:
        query = input("\nInsira sua consulta (ou 'exit' para sair): ")
        if query.lower() == 'exit':
            break
        perform_query(query, vectorizer, tfidf_matrix, lyrics)


if __name__ == "__main__":
    main()


Consulta: to sou seu

Resultados:
Arquivo: fred-e-gustavo - Tó Sou Seu Part. Wesley Safadão.txt, Pontuação: 800.3496341233813
Arquivo: fred-e-gustavo - Tô Só Seu.txt, Pontuação: 800.3496341233813
Arquivo: nando-reis - Monóico.txt, Pontuação: 100.31266847403462
Arquivo: filosofia-reggae - O Meu Coração É Seu.txt, Pontuação: 100.27056531934427
Arquivo: so-pra-contrariar - Ela É Jogo Duro.txt, Pontuação: 100.16071311878486

Consulta: to sou seu to sou seu

Resultados:
Arquivo: fred-e-gustavo - Tó Sou Seu Part. Wesley Safadão.txt, Pontuação: 400.3496341233813
Arquivo: fred-e-gustavo - Tô Só Seu.txt, Pontuação: 400.3496341233813
Arquivo: adriana-calcanhoto - Intimidade Sou Seu.txt, Pontuação: 0.7931469363384632
Arquivo: paulo-ricardo - Viver Um Grande Amor.txt, Pontuação: 0.7207289781475179
Arquivo: grupo-show-de-bola-2 - Anjo da Guarda.txt, Pontuação: 0.6712494619080783

Consulta: 

Resultados:


KeyboardInterrupt: 

In [31]:
#ver as palavras na matriz



